# import libraries

In [1]:
! pip install beautifulsoup4
! pip install lxml
! pip install requests
!pip install html5lib
import html.parser
from bs4 import BeautifulSoup
import requests
! pip install urllib3
import urllib3.request
! pip install geocoder

     |████████████████████████████████| 102kB 13.3MB/s 
     |████████████████████████████████| 81kB 17.7MB/s 


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


# scrap atlanta neighborhoods from wiki

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Atlanta").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

# get the data frame of atlanta neighborhood

In [4]:
# create a new DataFrame from the list
atlanta_df = pd.DataFrame({"Neighborhood": neighborhoodList})
print(atlanta_df.shape)
atlanta_df.head()

(123, 1)


,Neighborhood
0,Table of Atlanta neighborhoods by population
1,Adair Park
2,"Adamsville, Atlanta"
3,Ansley Park
4,Ardmore (Atlanta)


# drop first row,since it is not a neighborhood

In [5]:
atlanta_df.drop([0],axis=0,inplace=True)
atlanta_df.reset_index(drop=True,inplace=True)
atlanta_df.head()

,Neighborhood
0,Adair Park
1,"Adamsville, Atlanta"
2,Ansley Park
3,Ardmore (Atlanta)
4,"Argonne Forest, Atlanta"


# find the gographical coordinates for atlanta neighborhoods

In [6]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Atlanta, Georgia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [7]:
# call the function to get the coordinates, store in a new list using list comprehension
coords_atlanta = [ get_latlng(neighborhood) for neighborhood in atlanta_df["Neighborhood"].tolist() ]
coords_atlanta

[[33.72997000000004, -84.41043999999994],
 [33.761835, -84.51394993954038],
 [33.793120000000044, -84.37989999999996],
 [33.803572345194276, -84.39817429008188],
 [33.77992874387875, -84.3776505],
 [33.75043000000005, -84.42094999999995],
 [33.776050000000055, -84.35553999999996],
 [33.791540043857125, -84.39615992094332],
 [33.79323902306597, -84.50363485486446],
 [33.698790970580376, -84.50628092880167],
 [33.71710000000007, -84.36482999999998],
 [33.79929000000004, -84.41042999999996],
 [33.79210000000006, -84.42097999999999],
 [33.78022901957183, -84.49972276134567],
 [33.73703158970515, -84.3681735],
 [33.72458000000006, -84.36367999999999],
 [33.62608971539913, -84.55121424628494],
 [33.82572596008056, -84.3807059492887],
 [33.86434000000003, -84.42104999999998],
 [33.84700909207371, -84.3747865801192],
 [33.83839000000006, -84.38070999999997],
 [33.750290000000064, -84.36446999999998],
 [33.765990000000045, -84.34106999999995],
 [33.717250000000035, -84.40404999999998],
 [33.778

In [8]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords_atlanta, columns=['Latitude', 'Longitude'])

In [9]:
# merge the coordinates into the original dataframe
atlanta_df['Latitude'] = df_coords['Latitude']
atlanta_df['Longitude'] = df_coords['Longitude']

In [10]:
# check the neighborhoods and the coordinates
print(atlanta_df.shape)
atlanta_df.head()

(122, 3)


,Neighborhood,Latitude,Longitude
0,Adair Park,33.729970,-84.410440
1,"Adamsville, Atlanta",33.761835,-84.513950
2,Ansley Park,33.793120,-84.379900
3,Ardmore (Atlanta),33.803572,-84.398174
4,"Argonne Forest, Atlanta",33.779929,-84.377651


In [11]:
# save the csv file
# save the DataFrame as CSV file
atlanta_df.to_csv("atlanta_df.csv", index=False)

# map of atlanta with neighborhood superimposed on the top

In [12]:
# get the coordinates of Atlanta
address = 'Atlanta, Georgia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Atlanta, Georgia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Atlanta, Georgia 33.7490987, -84.3901849.


In [13]:
# create map of Atlanta using latitude and longitude values
map_atlanta = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(atlanta_df['Latitude'], atlanta_df['Longitude'], atlanta_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_atlanta)  
    
map_atlanta

In [14]:
# save the map as HTML file
map_atlanta.save('map_kl.html')

# use the foursquare API to explore and cluster the neighborhoods

In [15]:
# define Foursquare Credentials and Version
CLIENT_ID = 'PWAT3AVSYBTAROZQ1TLIZF1CI503F450IGVILXTI3M4LDF1W' # your Foursquare ID
CLIENT_SECRET = 'K1P1B42ANDEJI4NCUBNLJK3UYDYTKTKWIJDUMRQFSARBT1HD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PWAT3AVSYBTAROZQ1TLIZF1CI503F450IGVILXTI3M4LDF1W
CLIENT_SECRET:K1P1B42ANDEJI4NCUBNLJK3UYDYTKTKWIJDUMRQFSARBT1HD


# lets find the top 100 venues within 1500 meters

In [16]:
radius = 1500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(atlanta_df['Latitude'], atlanta_df['Longitude'], atlanta_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [17]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8528, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adair Park,33.72997,-84.41044,Adair Park One,33.730525,-84.412837,Park
1,Adair Park,33.72997,-84.41044,Monday Night Garage,33.729407,-84.418303,Brewery
2,Adair Park,33.72997,-84.41044,Boxcar Atl,33.730106,-84.418582,Gastropub
3,Adair Park,33.72997,-84.41044,Krispy Kreme Doughnuts,33.737829,-84.415880,Donut Shop
4,Adair Park,33.72997,-84.41044,Glitter's Fitness Club,33.736542,-84.402209,Gym / Fitness Center


## venues in each neighborhood

In [18]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adair Park,59,59,59,59,59,59
"Adamsville, Atlanta",29,29,29,29,29,29
Ansley Park,100,100,100,100,100,100
Ardmore (Atlanta),100,100,100,100,100,100
"Argonne Forest, Atlanta",100,100,100,100,100,100
Ashview Heights,68,68,68,68,68,68
Atkins Park,100,100,100,100,100,100
Atlantic Station,100,100,100,100,100,100
"Bankhead, Atlanta",22,22,22,22,22,22


## unique venucategories in atlanta neighborhoods

In [19]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 322 uniques categories.


In [20]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:25]

array(['Park', 'Brewery', 'Gastropub', 'Donut Shop',
       'Gym / Fitness Center', 'Trail', 'Vegetarian / Vegan Restaurant',
       'Caribbean Restaurant', 'Chinese Restaurant',
       'Fried Chicken Joint', 'Pizza Place', 'Seafood Restaurant',
       'Clothing Store', 'Museum', 'Cosmetics Shop', 'Bank',
       'Paper / Office Supplies Store', 'Discount Store', 'Video Store',
       'Dance Studio', 'Sandwich Place', 'Grocery Store', 'Gas Station',
       'Convenience Store', 'BBQ Joint'], dtype=object)

## cluster the venues based on the indian retaurant

In [21]:
# check if the results contain "indian restaurant"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

## analyze neighborhoods based on one hot coding

In [22]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()


(8528, 323)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,College Rec Center,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indoor Play Area,Insurance Office,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Piercing Parlor,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Storage Facility,Strip Club,

##mean of the frequency of occurence of each category on each neighborhood

In [23]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(121, 323)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,College Rec Center,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indoor Play Area,Insurance Office,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Piercing Parlor,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Storage Facility,Strip Club,

In [24]:
len(kl_grouped[kl_grouped["Indian Restaurant"] > 0])

41

## create a new dataframe based on indian restaurant data only

In [25]:
ind_rest = kl_grouped[["Neighborhoods","Indian Restaurant"]]
ind_rest.head()

,Neighborhoods,Indian Restaurant
0,Adair Park,0.00
1,"Adamsville, Atlanta",0.00
2,Ansley Park,0.01
3,Ardmore (Atlanta),0.00
4,"Argonne Forest, Atlanta",0.01


## run k-means clustering to divide the indian restaurna clusters into 4 clusters

In [26]:
# set number of clusters
kclusters = 4

ind_clustering = ind_rest.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ind_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 1, 0, 1, 1, 0, 0], dtype=int32)

In [27]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ind_merged = ind_rest.copy()

# add clustering labels
ind_merged["Cluster Labels"] = kmeans.labels_

In [28]:
ind_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
ind_merged.head()

,Neighborhood,Indian Restaurant,Cluster Labels
0,Adair Park,0.00,0
1,"Adamsville, Atlanta",0.00,0
2,Ansley Park,0.01,1
3,Ardmore (Atlanta),0.00,0
4,"Argonne Forest, Atlanta",0.01,1


In [29]:
# merge ind_merged with atlanta_df to add latitude/longitude for each neighborhood
ind_merged = ind_merged.join(atlanta_df.set_index("Neighborhood"), on="Neighborhood")

print(ind_merged.shape)
ind_merged.head() # check the last columns!

(121, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Adair Park,0.00,0,33.729970,-84.410440
1,"Adamsville, Atlanta",0.00,0,33.761835,-84.513950
2,Ansley Park,0.01,1,33.793120,-84.379900
3,Ardmore (Atlanta),0.00,0,33.803572,-84.398174
4,"Argonne Forest, Atlanta",0.01,1,33.779929,-84.377651


In [30]:
# sort the results by Cluster Labels
print(ind_merged.shape)
ind_merged.sort_values(["Cluster Labels"], inplace=True)
ind_merged.head()

(121, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Adair Park,0.0,0,33.729970,-84.410440
80,"Oakland City, Atlanta",0.0,0,33.719530,-84.428690
79,North Ormewood Park,0.0,0,33.743096,-84.355485
78,North Buckhead,0.0,0,33.862520,-84.365970
77,Neighborhoods in Atlanta,0.0,0,33.804370,-84.393420


## lets visualize the resulting clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ind_merged['Latitude'], ind_merged['Longitude'], ind_merged['Neighborhood'], ind_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## lets examine the clusters
**cluster 0**

In [33]:
# cluster 0
ind_merged.loc[ind_merged['Cluster Labels'] == 0]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Adair Park,0.0,0,33.729970,-84.410440
80,"Oakland City, Atlanta",0.0,0,33.719530,-84.428690
79,North Ormewood Park,0.0,0,33.743096,-84.355485
78,North Buckhead,0.0,0,33.862520,-84.365970
77,Neighborhoods in Atlanta,0.0,0,33.804370,-84.393420
75,Mozley Park,0.0,0,33.751160,-84.435380
74,Morningside/Lenox Park,0.0,0,33.798030,-84.359560
73,Midwest Cascade,0.0,0,33.726860,-84.533920
69,Marietta Street Artery,0.0,0,33.776150,-84.412880
67,"Loring Heights, Atlanta",0.0,0,33.737216,-84.351382


**cluster 1**

In [34]:
ind_merged.loc[ind_merged['Cluster Labels'] == 1]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
41,"Fairlie–Poplar, Atlanta",0.01,1,33.756698,-84.390520
42,"Five Points, Atlanta",0.01,1,33.776831,-84.389162
101,South Downtown,0.01,1,33.749108,-84.382910
51,History of Virginia–Highland,0.01,1,33.782250,-84.356110
50,Historic Brookhaven,0.01,1,33.854070,-84.357990
104,Technology Square (Atlanta),0.01,1,33.776860,-84.389570
105,Tenth Street (Atlanta),0.01,1,33.781788,-84.378312
7,Atlantic Station,0.01,1,33.791540,-84.396160
107,The Gulch (Atlanta),0.01,1,33.755940,-84.397320
99,"SoNo, Atlanta",0.01,1,33.748310,-84.391110


**cluster 2**

In [35]:
ind_merged.loc[ind_merged['Cluster Labels'] == 2]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
117,West Midtown,0.02,2,33.778512,-84.36994
71,Midtown Atlanta,0.02,2,33.778512,-84.36994


**cluster 3**

In [36]:
ind_merged.loc[ind_merged['Cluster Labels'] == 3]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
49,"High Point, Atlanta",0.015385,3,33.793965,-84.353292


**observation**
most of the indian resturant are concentrated in cluster 0 and cluster 1. Those areas are located in down town atlanta and east and north side of the downtown atlanta. clusters 2 and 3 do not have that many indian resturants. Based on the observations, it seems opening a new indian restaurant in cluster 2 and 3 would face less competition as comapared to cluster 0 and 1. 
